### Exercícios práticos 04 - Syanne Tavares

#### Resolução de sistemas lineares

- Podem ter 3 situações

1. Solução única -> retas concorrentes -> se tocam em 1 único ponto
2. Infinitas soluções -> retas coicidentes - pode ser qualquer tipo de valor
3. Sem solução -> Reta paralelas 

- 2 formas de resolver 
1. Métodos Diretos 
2. Métodos iterativos


## Exercício 01 - 

Implemente todas as técnicas de solução de sistemas lineares vistas em aula e faça uma comparação entre elas para as seguintes situações

# Métodos Diretos
## Fornecem soluções exatas com um número finito de operações. Podem ocorrer erros de arredondamento


# 1. Eliminação de Gauss
Resumidamente -> transforma o sistema em um sistema equivalente do tipo triangular superior e resolve a partir disso

In [4]:
import numpy as np
import pandas as pd

  
# Resolve o sistema triangular superior.
def SistemaTriangularSuperior(Ax,b):
  """Função que resolve um sistema linear triangular superior do tipo  𝐴𝑥 = b que recebe como parâmetro """
  n = len(b) #tamanho do vetor b
  x= [b[i]/Ax[i][i] if i == 0 else None for i in range(0,len(b))]

  x[n-1] = b[n-1]/Ax[n-1][n-1]

  for i in list(range(n-1,0,-1)):
    s = 0
    for j in list(range(i+1,n+1)):
      s = s + Ax[i-1][j-1]*x[j-1]

    x[i-1] = (b[i-1]-s)/(Ax[i-1][i-1])

  return x

def EliminacaoDeGauss(A,b):
  """Função que resolve um sistema linear do tipo Ax = b que recebe como parâmetro
     Return : retorna o vetor x resultante"""
  
  # ELIMINAÇÃO DE GAUSS
  n = len(b)

  # Calculo dos pivos.
  for k in range(1,n+1):
    # PIVOTEAMENTO PARCIAL
    for i in range(k+1, len(A)+1):
      if (abs(A[i-1][k-1]) > abs(A[k-1][k-1])):
        [A[k-1], A[i-1]] = [A[i-1], A[k-1]]
        [b[k-1], b[i-1]] = [b[i-1], b[k-1]]

    # Calculo dos multiplicadores.
    for i in list(range(k+1,n+1)):
      m = A[i-1][k-1]/A[k-1][k-1]
      A[i-1][k-1] = 0
      
      # Atualizar demais valores da linha
      for j in list(range(k+1,n+1)):
        A[i-1][j-1] = A[i-1][j-1]-m*A[k-1][j-1]
      b[i-1] = b[i-1] - m*b[k-1]

  # print("depois da eliminação\n Ax =  ",Ax2,"\nb = ",b2)
  x = SistemaTriangularSuperior(A,b)
  return(x)


In [5]:
Ax = [[3, 2, 4],
      [1, 1, 2],
      [4, 3,-2]]
b = [1, 2, 3]

print("X: ",EliminacaoDeGauss(Ax,b))

X:  [-3.0, 5.0, 0.0]


In [6]:
#exemplo 2
Ax = [[0,9, 8],
    [4, 5 ,6],
    [6, 4, 9]]

b =  [105, 83 ,93]
print("X: ",EliminacaoDeGauss(Ax,b))

X:  [4.999999999999999, 9.0, 3.000000000000001]


# Fatoração LU com pivoteamento parcial

#### O processo de fatoração consistem em decompor a matriz A em um produto de dois ou mais fatores
▪ De forma que seja mais fácil resolver o sistema em etapas

In [49]:
def formata_matriz(M):
  """Função que printa uma matriz formatando-a"""
  m = len(M) # número de linhas
  n = len(M[0]) # número de colunas
  s = ""
  for i in range(m):
      for j in range(n):
          s += "%9.3f " % M[i][j]
      s +=  "\n"
  return s

# Resolve o sistema triangular inferior.
def SistemaTriangularInferior(Ax,b):
  """Função que resolve um sistema triangular inferior"""
  n = len(b)
  
  y= [b[i]/Ax[i][i] if i == 0 else None for i in range(0,len(b))]

  for i in list(range(1,n+1,1)):
    s = 0
    for j in list(range(1,i,1)):
      s = s + Ax[i-1][j-1]*y[j-1]

    y[i-1] = b[i-1] - s

  return y

def LU_PivParc(A,b):
  """Função que resolve um sistema linear do tipo Ax = b usando o método de Fatoração LU com pivoteamento parcial 
     A = matriz de coeficientes
     b = vetor b resultante
     Retorna um vetor x calculado"""
  n = len(A)
  p=[0]*n
  

  ###### pivoteamento parcial 
  p = list(range(1,n+1)) #"matriz permutação"

  for k in range(1,n+1):
    pv = abs(A[k-1][k-1])
    Lpivo = k

    for i in range(k+1,n+1):
        if abs(A[i-1][k-1])> pv :
          pv = abs(A[i-1][k-1])
          Lpivo = i
  
    if pv == 0: 
      return None #matriz singular
      

    if Lpivo != k : # se for diferente então tem que realizar a permutação
      [p[k-1],p[Lpivo-1]] = [p[Lpivo-1],p[k-1]] # recurso guanabaristico
    
      for j in range(1,n+1):
          [A[k-1][j-1],A[Lpivo-1][j-1]] = [A[Lpivo-1][j-1],A[k-1][j-1]] 

  # resolução dos sistemas triangulares
  # Calculo dos pivos.
  for k in range(1,n+1):
    # Calculo dos multiplicadores.
    for i in range(k+1,n+1):
      m = A[i-1][k-1]/A[k-1][k-1]
      A[i-1][k-1] = m
      # Atualizar demais valores da linha
      for j in range(k+1,n+1,1):
        A[i-1][j-1] = A[i-1][j-1]-m*A[k-1][j-1]
  ### multiplicação da b pela "matriz permutação"
  c=[None for _ in range(0,len(b))]
  for i in range(1,n+1):
    Lpivo = p[i-1]
    c[i-1]=b[Lpivo-1]

  # resolução dos sistemas 
  y = SistemaTriangularInferior(A,c)
  x = SistemaTriangularSuperior(A,y)

  return x

In [50]:
# Teste com dados.
Ai = [[3, -4, 1],
      [1, 2, 2],
      [4, 0, -3]]

bi = [9, 3, -2]

print("Ai = \n",formata_matriz(Ai))
#Obtendo os fatores L e U.
print("X =")
print(LU_PivParc(Ai,bi))

Ai = 
     3.000    -4.000     1.000 
    1.000     2.000     2.000 
    4.000     0.000    -3.000 

X =
[1.0, -1.0, 2.0]


## Métodos iterativos

### 3. Gauss-Jacobi

In [9]:
import math
def TesteConvergencia(MatrizA):
    """Função que realiza teste de convergência pra uma matriz A
        Retorna True caso a seja convergente e False caso contrário"""
    Alfas=list()

    for i in range(len(MatrizA)):
        s=0
        for j in range(len(MatrizA)):
            if j!=i:
                s+=MatrizA[i][j]

        Alfas.append(s/MatrizA[i][i])
    # print(Alfas)
    # print(max(Alfas))

    if max(Alfas) < 1:
        return True
    else:
        return False
def criterioParada(x,xk,precisao):
    """Função que compara x com x-1 verificando a precisão do método como um critério de parada"""
    soma = 0
    zip_object = zip(x, xk) # une as informações de duas variáveis
    for list1_i, list2_i in zip_object:
        soma = soma + math.fabs(list1_i-list2_i)

    if (soma < precisao):
        return True
    else:
        return False   
            
def GaussJacobi(MatrizA,b,max_iteracoes,precisao):
    """Função que resolve um sistema linear do tipo Ax = b através do método iterativo de Gauss-Jacobi 
    Parâmetros:
        MatrizA = matriz de coeficientes do sistema linear
        b = vetor b
        max_iteracoes = npumero máximo de iterações
        precisao = precisão do erro
        
        retorno: tupla com
      vetor x resultante e com o número de iterações"""
    n= len(b) #tamanho de do vetor base

    X=b.copy() # copiando b pra x como chute inicial
    solucao = True
    # gerar chute inicial
    for i in list(range(1,n+1)): # percorre cada elemento de x de 1 até n
        # dividir pelo elemento da diagonal principal
        if math.fabs(MatrizA[i-1][i-1])> 0 :# verifica se é possível realizar divisão se a diagonal é diferente de zero
            X[i-1] = b[i-1]/MatrizA[i-1][i-1]
        else:
            solucao = False # não se pode calcular o chute inicial
            break
    if solucao and TesteConvergencia(MatrizA):
        # print("Iteração 0")
        # print("x = ",X)

        xk = X.copy()

        iter = 0
        
        # realiza o calculo de uma função iteração
        while (iter < max_iteracoes): # início das iterações
            iter = iter + 1
            for i in list(range(1,n+1)):
                s = 0

                #realiza o somatório
                for j in list(range(1,n+1)):
                    if ((i-1) != (j-1)):
                        s = s + MatrizA[i-1][j-1]*X[j-1] # somando os termos que não são da diagonal

                xk[i-1] = (1/MatrizA[i-1][i-1])*(b[i-1]-s) # equivale a 1/a(ii)*(bi - somatório de aij*xj)
                
            # print("Iteração: ",iter)
            # print("xk = ",xk)

            #verifica o critério de parada ->  O processo é repetido até ue x^k e x^k-1 eestejam suficientemente próximos
            if criterioParada(X,xk,precisao):
                X = xk.copy()
                break    
            X = xk.copy()
      
        return X,iter
    else:
        return None,None


In [10]:
#exemplo 

A = [[10, 2,  1],
     [ 1, 5,  1],
     [ 2, 3, 10]]

b = [7, -8, 6]

x,iter = GaussJacobi(A,b,10,0.01)
print("x = ",x)
print("N iterações = ",iter)



x =  [1.000236, -1.9989360000000003, 1.000284]
N iterações =  5


### 4. Gauss-Seidel

In [11]:
def GaussSeidel(MatrizA,b,max_iteracoes,precisao):
    """Função que resolve um sistema linear do tipo Ax = b através do método iterativo de Gauss-Seidel
      Parâmetros:
      MatrizA = matriz de coeficientes do sistema linear
      b = vetor b
      max_iteracoes = npumero máximo de iterações
      precisao = precisão do erro
      
      retorno: tupla com
      vetor x resultante e com o número de iterações """
    n= len(b) #tamanho de do vetor base

    X=b.copy() # copiando b pra x como chute inicial
    solucao = True
    # gerar chute inicial
    for i in list(range(1,n+1)): # percorre cada elemento de x de 1 até n
        # dividir pelo elemento da diagonal principal
        if math.fabs(MatrizA[i-1][i-1])> 0 :# verifica se é possível realizar divisão se a diagonal é diferente de zero
            X[i-1] = b[i-1]/MatrizA[i-1][i-1]
        else:
            solucao = False # não se pode calcular o chute inicial
            break
    if solucao and TesteConvergencia(MatrizA):
      # print("Iteração 0")
      # print("x = ",X)

      xk = X.copy()

      iter    = 0
      
      # realiza o calculo de uma função iteração
      while (iter < max_iteracoes): # início das iterações
        iter = iter + 1
        for i in list(range(1,n+1)):
          s = 0

          #realiza o somatório
          for j in list(range(1,n+1)):
            if ((i-1) > (j-1)): # se o índice da linha é maior que o da coluna
              s = s + MatrizA[i-1][j-1]*xk[j-1] # usa o xk que tem os valores atualizados
            elif ((i-1) < (j-1)): # se o índice da linha é maior que o da coluna
              s = s + MatrizA[i-1][j-1]*X[j-1] 

          xk[i-1] = (1/MatrizA[i-1][i-1])*(b[i-1]-s) # equivale a 1/a(ii)*(bi - somatório de aij*xj)
      
        # print("Iteração: ",iter)
        # print("xk = ",xk)

        #verifica o critério de parada ->  O processo é repetido até ue x^k e x^k-1 eestejam suficientemente próximos
        if criterioParada(X,xk,precisao):
          X = xk.copy()
          break    
        X = xk.copy()
     
      return X,iter
    else:
       return None,None

In [12]:
#exemplo 

A = [[10, 2,  1],
     [ 1, 5,  1],
     [ 2, 3, 10]]

b = [7, -8, 6]

x,iter = GaussSeidel(A,b,10,0.01)
print("x = ",x)
print("N iterações = ",iter)


x =  [0.9999577430400001, -2.000046123328, 1.0000222883904002]
N iterações =  4


## Comparação entre os métodos

In [100]:
def mudancaAleatoria(mat):

    n = mat.shape[0] 
    lin = np.random.randint(n)

    if(np.random.rand()>0.25):
        mat[lin,:] +=np.random.rand()*mat[np.random.randint(n),:] # 
    else:
        mat[lin,:]-=np.random.rand()*mat[np.random.randint(n),:]

def gerarSistema(n):
    mat = np.zeros((n,n+1)) # cria uma matriz zerada de dimensão n
    # print(mat)
    b= np.random.random(n) # gera um b aleatório
    mat[:,n]= b # adiciona esse b como última coluna da matriz
    # print(mat)
    mat[:,:-1] = np.identity(n) # cria uma matriz identidade 
    # print(mat)
    mat * np.random.randint(50,100)
    # print(mat)

    rounds = np.random.randint(9,13)
    while(rounds>0) :
        mudancaAleatoria(mat)
        rounds-=1
        # print("mudou")
        # print(mat)
    A = mat[:,:-1].tolist()
    x = b.tolist()
    newb = [element[0] for element in mat[: , -1:].tolist()]

    print("Exemplos: ")
    print("Matriz: \n",A)
    print("x = ",x)
    print("b = ",newb)

    return(A,x,newb)

In [114]:
def GetExemploSistemasLineares(Dimensao_n,it_max=300,precisao=0.1,metodos=[1,2,3,4]):
    """FUNÇÃO QUE GERA UM EXEMPLO DE SISTEMA LINEAR DE DIMENSÃO RECBIDA POR PARÂMETRO E RESOLVE POR MÉTODOS DIRETOS E ITERATIVOS COMPARANDO OS RESULTADOS\n
        Parâmetros\n
        Dimensao_n = Dimensão da matriz de exemplo\n
        it_max : número máximo de iterações pros métodos iterativos\n
        precisao: precisão a ser utilizada nos métodos iterativos\n
        metodos : lista com os números dos métodos a serem utilizados [1(Eliminação de Gauss),2(LU com piv parc),3(Gauss-Jacobi),4(Gauss-Seidel)]
        """

    Ax,x,b = gerarSistema(Dimensao_n)
    results={"X":x}
    if 1 in metodos:
          results["Eliminação de Gauss"]=EliminacaoDeGauss(Ax,b)
    if 2 in metodos:
          results["Fatoração LU e Piv Parc:"]=LU_PivParc(Ax,b)
    if 3 in metodos:        
        resultJacobi,iteracoes = GaussJacobi(Ax,b,it_max,precisao)
        results[f"Gauss-Jacobi {iteracoes} iterações:"] = resultJacobi
    if 4 in metodos:  
        resultSeidel,iteracoes = GaussSeidel(Ax,b,it_max,precisao)
        results[f"Gauss-Seidel {iteracoes} iterações:"]=resultSeidel

    print("\nCOMPARAÇÃO RESULTADOS: \n")

#     print("X por eliminação de gauss: ",EliminacaoDeGauss(Ax,b))
#     print("X por Fatoração LU com pivoteamento parcial: ",LU_PivParc(Ax,b))
#     print(f"X por Gauss-Jacobi: {resultJacobi}, com {iteracoes} iterações") 
#     print(f"X por Gauss-Seidel: {resultSeidel}, com {iteracoes2} iterações")

    tabela = pd.DataFrame(results)
    display(tabela)

### Exemplo 1 -> n = 10

In [115]:
GetExemploSistemasLineares(10)

Exemplos: 
Matriz: 
 [[1.8020181864615803, -0.9677716563773878, 0.0, 0.0, 0.0, 0.0, 0.0, 0.6801294778172008, -0.0704063789839048, 0.274180819671503], [0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], [0.0, 0.0, 1.793617426612673, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], [0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.8842572706462802, 0.0, 0.0, 0.0], [0.0, -0.416228380511904, 0.0, 0.0, 1.0, 0.0, 0.0, -0.7476458666102822, 0.0, 0.0], [-0.10606714817043805, 0.09638476423939715, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0], [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0], [0.0, 0.5567186272279188, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, -0.03605587842093679, 0.1404110031235728], [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0], [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, -0.25678812642058235, 1.0]]
x =  [0.29895071769097636, 0.6525021388438009, 0.9149381041855168, 0.20406541163169878, 0.8358006683597196, 0.00019810349739968203, 0.21860450969233136, 0.6810664224770966, 0.23316366403860778, 0.09662706717128067]
b =  [0.38

,X,Eliminação de Gauss,Fatoração LU e Piv Parc:,Gauss-Jacobi 2 iterações:,Gauss-Seidel 2 iterações:
0,0.298951,0.298951,0.298951,0.298951,0.298951
1,0.652502,0.652502,0.652502,0.652502,0.652502
2,0.914938,0.914938,0.914938,0.914938,0.914938
3,0.204065,0.204065,0.204065,0.204065,0.204065
4,0.835801,0.835801,0.835801,0.835801,0.835801
5,0.000198,0.000198,0.000198,0.000198,0.000198
6,0.218605,0.218605,0.218605,0.218605,0.218605
7,0.681066,0.681066,0.681066,0.681066,0.681066
8,0.233164,0.233164,0.233164,0.233164,0.233164
9,0.096627,0.096627,0.096627,0.096627,0.096627


In [116]:
#apnas métodos diretos
GetExemploSistemasLineares(10,metodos=[1,2])

Exemplos: 
Matriz: 
 [[1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.5442797874891147], [0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], [0.0, 0.0, 0.10699257934871764, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.09022801396207691], [0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], [0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, -0.4641654123222929, -0.9442671009154664, 0.0], [0.0, 0.0, 0.8412615711496595, 0.0, 0.3958377783143492, 1.5231514374217772, 0.0, -0.18373420558402026, 0.0, 0.0], [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0], [0.0, 0.22949459630216362, 0.0, 0.7387937809136889, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0], [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0], [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0]]
x =  [0.4537935659868697, 0.8535126626112758, 0.06703749406649928, 0.25451876020512887, 0.21021422904492093, 0.8694356107893375, 0.09272108723611683, 0.47273227131779916, 0.2725303679823148, 0.11938169739280036]
b =  [0.5187706108739129, 0.8535126626112758, 0.01794408786242313, 0.25451

,X,Eliminação de Gauss,Fatoração LU e Piv Parc:
0,0.453794,0.453794,0.453794
1,0.853513,0.853513,0.853513
2,0.067037,0.067037,0.067037
3,0.254519,0.254519,0.254519
4,0.210214,0.210214,0.210214
5,0.869436,0.869436,0.869436
6,0.092721,0.092721,0.092721
7,0.472732,0.472732,0.472732
8,0.272530,0.272530,0.272530
9,0.119382,0.119382,0.119382
